# Amber Prep Server - General Structure Test

This notebook tests the generalized `parse_structure` tool that can process:
- PDB database files (mmCIF or PDB format)
- Boltz-2/AlphaFold prediction outputs
- Any standard structure file

## Test Case: 1AKE (Adenylate Kinase)

- **PDB ID**: 1AKE
- **Chains**: A and B (homodimer)
- **Ligand**: AP5 (Bis(adenosine)-5'-pentaphosphate)
- **Goal**: Extract chain A + its bound AP5 ligand, prepare for MD


In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

from pathlib import Path
import json
import asyncio

# PDB ID to test
PDB_ID = "1AKE"
SELECT_CHAINS = ["A"]  # Chain A contains both protein and AP5 ligand

print(f"Test configuration:")
print(f"  PDB ID: {PDB_ID}")
print(f"  Chains: {SELECT_CHAINS}")
print(f"  Note: Using PDB format for simple chain selection")


In [ ]:
# Verify dependencies
print("Checking dependencies...")

required = ["gemmi", "rdkit", "dimorphite_dl", "pdbfixer"]
for pkg in required:
    try:
        __import__(pkg.replace("-", "_"))
        print(f"  ✓ {pkg}")
    except ImportError:
        print(f"  ✗ {pkg}")

# Check AmberTools
print("\nAmberTools:")
from common.base import BaseToolWrapper
for tool in ["antechamber", "parmchk2", "pdb4amber", "tleap", "packmol-memgen"]:
    wrapper = BaseToolWrapper(tool)
    print(f"  {'✓' if wrapper.is_available() else '✗'} {tool}")


## Step 1: Fetch Structure from PDB

Use `fetch_molecules` to download the structure from RCSB PDB.


In [ ]:
# Import structure server module
import importlib
import servers.structure_server as structure_module
importlib.reload(structure_module)

# Get fetch_molecules async function (direct access, no .fn needed)
fetch_molecules = structure_module.fetch_molecules

print("Fetching structure from PDB")
print("=" * 60)
print(f"PDB ID: {PDB_ID}")
print()

# Fetch the structure in PDB format (prefer_format="pdb" is default)
# PDB format uses simple chain IDs (A, B, C) which are intuitive
# mmCIF uses label_asym_id which assigns unique IDs per molecular entity
fetch_result = await fetch_molecules(pdb_id=PDB_ID, source="pdb", prefer_format="pdb")

if fetch_result["success"]:
    structure_file = fetch_result["file_path"]
    print(f"✓ Downloaded: {Path(structure_file).name}")
    print(f"✓ Format: {fetch_result['file_format']}")
    print(f"✓ Atoms: {fetch_result['num_atoms']}")
    print(f"✓ Chains: {fetch_result['chains']}")
else:
    print(f"✗ Error: {fetch_result['errors']}")
    raise RuntimeError("Failed to fetch structure")


## Step 2: Prepare Complex

Use `prepare_complex` to:
1. Inspect and split the structure into chains
2. Clean protein chains (PDBFixer + pdb4amber)
3. Prepare ligands (SMILES template matching + pH protonation)
4. Parameterize ligands (antechamber GAFF2 + AM1-BCC)
5. Merge all prepared structures into a single PDB file


In [ ]:
# Get prepare_complex function
prepare_complex = structure_module.prepare_complex

print("Preparing complex (clean + parameterize)")
print("=" * 60)
print(f"Input: {structure_file}")
print(f"Chains: {SELECT_CHAINS}")
print()

# With PDB format and use_author_chains=True (default):
# - select_chains=['A'] includes all molecules in chain A
# - This automatically includes both protein AND AP5 ligand
print(f"Processing chain(s): {SELECT_CHAINS}")

# Prepare the complex (this may take a few minutes for antechamber)
# SMILES for AP5 will be automatically fetched from PDB CCD
# By default, include_types=["protein", "ligand", "ion"] (water excluded)
complex_result = prepare_complex(
    structure_file=structure_file,
    select_chains=SELECT_CHAINS,  # Now works with simple chain IDs
    ph=7.4,
    process_proteins=True,
    process_ligands=True,
    run_parameterization=True
)

if complex_result["success"]:
    output_dir = Path(complex_result["output_dir"])
    print(f"✓ Job ID: {complex_result['job_id']}")
    print(f"✓ Output: {output_dir}")
    
    # Show protein results
    print(f"\n--- Proteins ({len(complex_result['proteins'])}) ---")
    for p in complex_result["proteins"]:
        status = "✓" if p["success"] else "✗"
        print(f"  {status} Chain {p['chain_id']}: {Path(p['output_file']).name}")
    
    # Show ligand results
    print(f"\n--- Ligands ({len(complex_result['ligands'])}) ---")
    for lig in complex_result["ligands"]:
        status = "✓" if lig["success"] else "✗"
        if lig["success"]:
            print(f"  {status} {lig['ligand_id']}: charge={lig['net_charge']}")
            print(f"      mol2: {Path(lig['mol2_file']).name}")
            print(f"      frcmod: {Path(lig['frcmod_file']).name}")
            if lig.get('pdb_file'):
                print(f"      pdb: {Path(lig['pdb_file']).name}")
        else:
            print(f"  {status} {lig.get('ligand_id', 'unknown')}: FAILED")
    
    # Show merge results
    print(f"\n--- Merged Structure ---")
    if complex_result.get("merged_pdb"):
        print(f"  ✓ {Path(complex_result['merged_pdb']).name}")
    else:
        print(f"  ✗ Not available")
    
    if complex_result["warnings"]:
        print(f"\n⚠ Warnings: {complex_result['warnings'][:3]}")  # Show first 3 warnings
else:
    print(f"✗ Error: {complex_result['errors']}")
    raise RuntimeError("Complex preparation failed")


## Step 3: Use Merged Structure

`prepare_complex` now automatically merges all prepared structures into a single PDB.
The merged PDB is available as `complex_result["merged_pdb"]`.

In [ ]:
# Import solvation server
import servers.solvation_server as solvation_module
importlib.reload(solvation_module)

solvate_structure = solvation_module.solvate_structure

# Use the merged PDB from prepare_complex (automatically created)
print("Using Merged Structure from prepare_complex")
print("=" * 60)

if complex_result.get("merged_pdb"):
    merged_pdb = complex_result["merged_pdb"]
    print(f"✓ Merged PDB: {Path(merged_pdb).name}")
    
    merge_info = complex_result.get("merge_result", {})
    if merge_info.get("statistics"):
        stats = merge_info["statistics"]
        print(f"✓ Total atoms: {stats.get('total_atoms', 'N/A')}")
        print(f"✓ Total residues: {stats.get('total_residues', 'N/A')}")
        print(f"✓ Total chains: {stats.get('total_chains', 'N/A')}")
else:
    print("✗ No merged PDB found in complex_result")
    print("  This may happen if no structures were successfully prepared.")
    raise RuntimeError("No merged PDB available")

## Visualize Merged Complex

Visualize the protein-ligand complex before solvation.

In [ ]:
# Visualize merged complex with py3Dmol
try:
    import py3Dmol
except ImportError:
    %pip install py3Dmol
    import py3Dmol

print("Visualizing merged complex (before solvation)")
print("=" * 60)

# Read the merged PDB
with open(merged_pdb, 'r') as f:
    pdb_content = f.read()

# Count atoms
lines = pdb_content.split('\n')
atom_lines = [l for l in lines if l.startswith('ATOM') or l.startswith('HETATM')]
print(f"Total atoms: {len(atom_lines)}")

# Create viewer
view = py3Dmol.view(width=800, height=500)
view.addModel(pdb_content, 'pdb')

# Style: protein cartoon, ligand sticks
AMINO_ACIDS = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU', 
               'GLY', 'HIS', 'HID', 'HIE', 'HIP', 'ILE', 'LEU', 'LYS',
               'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']

# Protein - cartoon
view.setStyle({'resn': AMINO_ACIDS}, {'cartoon': {'color': 'spectrum'}})

# Ligands - sticks (anything else)
ligand_resnames = [lig['ligand_id'] for lig in complex_result['ligands'] if lig['success']]
for resn in ligand_resnames:
    view.setStyle({'resn': resn}, {'stick': {'color': 'green', 'radius': 0.3}})
    view.addResLabels({'resn': resn}, {'fontSize': 12, 'fontColor': 'white', 
                                        'backgroundColor': 'green', 'backgroundOpacity': 0.8})

view.zoomTo()
view.setProjection('orthographic')

print(f"\n🔹 Protein: Cartoon (spectrum)")
print(f"🔹 Ligands: {ligand_resnames} (green sticks)")

view.show()

## Solvate Structure

Add water box and ions using packmol-memgen.

In [ ]:
# Solvate the merged structure (use absolute paths)
print("Solvating...")
print("=" * 60)
solvate_result = solvate_structure(
    pdb_file=str(Path(merged_pdb).resolve()),
    output_dir=str(output_dir.resolve()),
    output_name="solvated",
    dist=12.0,
    cubic=True,
    salt=True,
    saltcon=0.15
)

if solvate_result["success"]:
    solvated_pdb = solvate_result["output_file"]
    print(f"✓ Solvated: {Path(solvated_pdb).name}")
    print(f"✓ Atoms: {solvate_result['statistics'].get('total_atoms', 'N/A')}")
else:
    print(f"✗ Solvation failed: {solvate_result['errors']}")
    raise RuntimeError("Solvation failed")

## Step 4: 3D Visualization

Visualize the solvated system with py3Dmol.


In [ ]:
# 3D Visualization with py3Dmol
try:
    import py3Dmol
except ImportError:
    print("Installing py3Dmol...")
    %pip install py3Dmol
    import py3Dmol

print("Visualizing solvated system")
print("=" * 60)

# Read the solvated PDB
with open(solvated_pdb, 'r') as f:
    pdb_content = f.read()

# Count atoms by type for info
lines = pdb_content.split('\n')
atom_lines = [l for l in lines if l.startswith('ATOM') or l.startswith('HETATM')]
print(f"Total atoms: {len(atom_lines)}")

# Create viewer
view = py3Dmol.view(width=900, height=600)
view.addModel(pdb_content, 'pdb')

# Style: protein cartoon, ligand sticks, water dots
AMINO_ACIDS = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU', 
               'GLY', 'HIS', 'HID', 'HIE', 'HIP', 'ILE', 'LEU', 'LYS',
               'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
WATER = ['WAT', 'HOH']
IONS = ['NA', 'CL', 'Na+', 'Cl-', 'K', 'K+']

# Protein - cartoon
view.setStyle({'resn': AMINO_ACIDS}, {'cartoon': {'color': 'spectrum'}})

# Water - small spheres
view.setStyle({'resn': WATER}, {'sphere': {'radius': 0.15, 'color': 'lightblue'}})

# Ions - spheres
view.setStyle({'resn': ['NA', 'Na+']}, {'sphere': {'radius': 0.8, 'color': 'purple'}})
view.setStyle({'resn': ['CL', 'Cl-']}, {'sphere': {'radius': 0.8, 'color': 'yellow'}})

# Ligands - sticks (anything else)
ligand_resnames = [lig['ligand_id'] for lig in complex_result['ligands'] if lig['success']]
for resn in ligand_resnames:
    view.setStyle({'resn': resn}, {'stick': {'color': 'green', 'radius': 0.3}})
    view.addResLabels({'resn': resn}, {'fontSize': 12, 'fontColor': 'white', 
                                        'backgroundColor': 'green', 'backgroundOpacity': 0.8})

view.zoomTo()
view.setProjection('orthographic')

print(f"\n🔹 Protein: Cartoon (spectrum)")
print(f"🔹 Ligands: {ligand_resnames} (green sticks)")
print(f"🔹 Water: Dots (light blue)")
print(f"🔹 Ions: Spheres (Na+=purple, Cl-=yellow)")

view.show()


## Step 5: Build Amber System (tleap)

Generate Amber topology (parm7) and coordinate (rst7) files using tleap.
These files can be used with OpenMM's AmberPrmtopFile/AmberInpcrdFile for more precise force field handling.


In [ ]:
# Import amber server module
import servers.amber_server as amber_module
importlib.reload(amber_module)

build_amber_system = amber_module.build_amber_system

# Collect ligand parameters from prepare_complex result
ligand_params = []
for lig in complex_result.get("ligands", []):
    if lig.get("success") and lig.get("mol2_file"):
        ligand_params.append({
            "mol2": lig["mol2_file"],
            "frcmod": lig["frcmod_file"],
            "residue_name": lig["ligand_id"][:3].upper()
        })

print(f"Ligand parameters: {len(ligand_params)} ligand(s)")
for i, lp in enumerate(ligand_params):
    print(f"  {i+1}. {lp['residue_name']}: {Path(lp['mol2']).name}")

# Build Amber system from solvated structure
print("\nBuilding Amber system...")
amber_result = build_amber_system(
    pdb_file=solvate_result["output_file"],
    ligand_params=ligand_params if ligand_params else None,
    box_dimensions=solvate_result.get("box_dimensions"),
    water_model="tip3p",
    output_name="system"
)

print(f"\nSuccess: {amber_result['success']}")
print(f"Solvent type: {amber_result['solvent_type']}")
if amber_result['success']:
    print(f"Topology: {amber_result.get('parm7')}")
    print(f"Coordinates: {amber_result.get('rst7')}")
    if amber_result.get("statistics"):
        stats = amber_result['statistics']
        if stats.get('num_atoms'):
            print(f"Atoms: {stats['num_atoms']}")
        if stats.get('num_residues'):
            print(f"Residues: {stats['num_residues']}")
else:
    print(f"Errors: {amber_result.get('errors')}")

if amber_result.get("warnings"):
    print(f"\nWarnings ({len(amber_result['warnings'])}):")
    for w in amber_result['warnings'][:5]:
        print(f"  - {w[:100]}")


## Step 6: OpenMM Simulation (Minimize → Equilibrate → Production)

Run a minimal MD simulation with OpenMM using Amber parm7/rst7 files from Step 5.
- Platform: CUDA > OpenCL (Mac GPU) > CPU
- Ensemble: NPT (1 atm, 300 K)
- Short run for testing

In [ ]:
# OpenMM Simulation: Minimize → Equilibrate → Production
# Uses Amber parm7/rst7 files from Step 5 (tleap)
import time

try:
    import openmm as mm
    from openmm import app, unit
    from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, StateDataReporter, DCDReporter, PDBFile
except ImportError:
    print("Installing OpenMM...")
    %pip install openmm
    import openmm as mm
    from openmm import app, unit
    from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, StateDataReporter, DCDReporter, PDBFile

def select_platform():
    """Select best available platform: CUDA > OpenCL > CPU"""
    platform_preference = ['CUDA', 'OpenCL', 'CPU']
    
    print("Checking available platforms...")
    for name in platform_preference:
        try:
            platform = mm.Platform.getPlatformByName(name)
            if name == 'CUDA':
                try:
                    platform.getPropertyDefaultValue('DeviceIndex')
                    print(f"  ✓ {name} available")
                    return platform, name
                except Exception:
                    print(f"  ✗ {name} not available (no GPU)")
                    continue
            elif name == 'OpenCL':
                print(f"  ✓ {name} available (Mac GPU)")
                return platform, name
            else:
                print(f"  ✓ {name} available")
                return platform, name
        except Exception as e:
            print(f"  ✗ {name} not available: {e}")
    
    raise RuntimeError("No suitable platform found!")

# Check if we have Amber files from Step 5
if 'amber_result' in dir() and amber_result.get('success') and amber_result.get('parm7'):
    parm7_file = amber_result['parm7']
    rst7_file = amber_result['rst7']
    
    print("=" * 60)
    print("OpenMM MD Simulation (using Amber parm7/rst7)")
    print("=" * 60)
    print(f"Topology: {Path(parm7_file).name}")
    print(f"Coordinates: {Path(rst7_file).name}")
    
    # Select platform
    platform, platform_name = select_platform()
    print(f"\n→ Using platform: {platform_name}")
    
    # Simulation parameters
    temperature = 300 * unit.kelvin
    pressure = 1 * unit.atmosphere
    timestep = 2 * unit.femtoseconds
    friction = 1 / unit.picosecond
    
    # Short runs for testing
    minimize_max_iter = 500
    equil_steps = 2500      # 5 ps equilibration
    prod_steps = 50000       # 100 ps production
    report_interval = 500   # Report every 1 ps
    
    print(f"\nSimulation parameters:")
    print(f"  Temperature: {temperature}")
    print(f"  Pressure: {pressure}")
    print(f"  Timestep: {timestep}")
    print(f"  Equilibration: {equil_steps} steps ({equil_steps * 2 / 1000} ps)")
    print(f"  Production: {prod_steps} steps ({prod_steps * 2 / 1000} ps)")
    
    # Load Amber files
    print(f"\n[Step 1] Loading Amber topology and coordinates...")
    t0 = time.time()
    prmtop = AmberPrmtopFile(parm7_file)
    inpcrd = AmberInpcrdFile(rst7_file)
    print(f"  ✓ Loaded in {time.time() - t0:.1f}s")
    print(f"  Atoms: {prmtop.topology.getNumAtoms()}")
    
    # Create system from Amber topology
    print(f"\n[Step 2] Creating OpenMM system from Amber topology...")
    t0 = time.time()
    
    # Create system with PME for explicit solvent
    system = prmtop.createSystem(
        nonbondedMethod=app.PME,
        nonbondedCutoff=10 * unit.angstrom,
        constraints=app.HBonds,
        rigidWater=True
    )
    
    # Add barostat for NPT
    system.addForce(mm.MonteCarloBarostat(pressure, temperature, 25))
    print(f"  ✓ System created in {time.time() - t0:.1f}s")
    
    # Create integrator and simulation
    integrator = mm.LangevinMiddleIntegrator(temperature, friction, timestep)
    simulation = Simulation(prmtop.topology, system, integrator, platform)
    
    # Set positions (and box vectors if present)
    simulation.context.setPositions(inpcrd.positions)
    if inpcrd.boxVectors is not None:
        simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
    
    # Energy minimization
    print(f"\n[Step 3] Energy minimization (max {minimize_max_iter} steps)...")
    t0 = time.time()
    state_before = simulation.context.getState(getEnergy=True)
    energy_before = state_before.getPotentialEnergy().value_in_unit(unit.kilojoules_per_mole)
    
    simulation.minimizeEnergy(maxIterations=minimize_max_iter)
    
    state_after = simulation.context.getState(getEnergy=True)
    energy_after = state_after.getPotentialEnergy().value_in_unit(unit.kilojoules_per_mole)
    print(f"  ✓ Minimized in {time.time() - t0:.1f}s")
    print(f"  Energy: {energy_before:.1f} → {energy_after:.1f} kJ/mol")
    
    # Initialize velocities
    simulation.context.setVelocitiesToTemperature(temperature)
    
    # Setup reporters
    dcd_file = output_dir / "trajectory.dcd"
    log_file = output_dir / "simulation.log"
    
    simulation.reporters.append(DCDReporter(str(dcd_file), report_interval))
    simulation.reporters.append(StateDataReporter(
        str(log_file), report_interval,
        step=True, time=True, potentialEnergy=True, kineticEnergy=True,
        totalEnergy=True, temperature=True, volume=True, density=True,
        speed=True
    ))
    simulation.reporters.append(StateDataReporter(
        sys.stdout, report_interval,
        step=True, time=True, temperature=True, speed=True, remainingTime=True,
        totalSteps=equil_steps + prod_steps
    ))
    
    # Equilibration
    print(f"\n[Step 4] NPT Equilibration ({equil_steps * 2 / 1000} ps)...")
    t0 = time.time()
    simulation.step(equil_steps)
    print(f"  ✓ Equilibration done in {time.time() - t0:.1f}s")
    
    # Production
    print(f"\n[Step 5] Production ({prod_steps * 2 / 1000} ps)...")
    t0 = time.time()
    simulation.step(prod_steps)
    print(f"  ✓ Production done in {time.time() - t0:.1f}s")
    
    # Save final state
    final_pdb = output_dir / "final_state.pdb"
    state = simulation.context.getState(getPositions=True, getVelocities=True)
    with open(final_pdb, 'w') as f:
        PDBFile.writeFile(simulation.topology, state.getPositions(), f)
    print(f"\n✓ Final state saved: {final_pdb.name}")
    
    # Summary
    print(f"\n{'='*60}")
    print("SIMULATION COMPLETE")
    print(f"{'='*60}")
    print(f"  Output directory: {output_dir}")
    print(f"  Trajectory: {dcd_file.name}")
    print(f"  Log: {log_file.name}")
    print(f"  Final PDB: {final_pdb.name}")
    
else:
    print("No solvated structure available. Run solvation step first.")

## Step 6: Trajectory Visualization with py3Dmol

Visualize the MD trajectory (equilibration + production) with py3Dmol animation.

In [ ]:
# Trajectory visualization with py3Dmol
import numpy as np
import tempfile

try:
    import mdtraj as md
except ImportError:
    print("Installing MDTraj...")
    %pip install mdtraj
    import mdtraj as md

import py3Dmol

# Check if we have trajectory files
if 'output_dir' in dir() and (output_dir / "trajectory.dcd").exists():
    dcd_file = output_dir / "trajectory.dcd"
    
    print("Loading trajectory...")
    
    # Load trajectory with MDTraj (use solvated PDB as topology)
    traj = md.load(str(dcd_file), top=solvated_pdb)
    print(f"  Frames: {traj.n_frames}")
    print(f"  Atoms: {traj.n_atoms}")
    print(f"  Time: {traj.time[0]:.1f} - {traj.time[-1]:.1f} ps")
    
    # Select protein and ligand atoms (exclude water and ions)
    protein_indices = traj.topology.select('protein')
    
    # Find ALL ligand residues (any non-standard non-water residue)
    lig_indices = []
    ligand_resnames = set()
    standard_res = {'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU', 
                    'GLY', 'HIS', 'HID', 'HIE', 'HIP', 'ILE', 'LEU', 'LYS', 
                    'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL',
                    'WAT', 'HOH', 'NA', 'CL', 'Na+', 'Cl-'}
    for residue in traj.topology.residues:
        if residue.name not in standard_res:
            atom_indices = [atom.index for atom in residue.atoms]
            lig_indices.extend(atom_indices)
            ligand_resnames.add(residue.name)
    lig_indices = np.array(lig_indices) if lig_indices else np.array([], dtype=int)
    
    print(f"  Protein atoms: {len(protein_indices)}")
    print(f"  Ligand atoms: {len(lig_indices)}")
    print(f"  Ligand types: {ligand_resnames if ligand_resnames else 'None'}")
    
    # Combine protein + ligand
    if len(lig_indices) > 0:
        keep_indices = np.concatenate([protein_indices, lig_indices])
    else:
        keep_indices = protein_indices
    keep_indices = np.unique(keep_indices)
    keep_indices = keep_indices[keep_indices < traj.n_atoms]
    
    # Subset trajectory to protein + ligand only
    traj_subset = traj.atom_slice(keep_indices)
    print(f"  Visualization atoms: {traj_subset.n_atoms}")
    
    # Sample frames for visualization
    max_frames = 15
    if traj_subset.n_frames > max_frames:
        frame_indices = np.linspace(0, traj_subset.n_frames - 1, max_frames, dtype=int)
        traj_viz = traj_subset[frame_indices]
        print(f"  Sampled {max_frames} frames for visualization")
    else:
        traj_viz = traj_subset
    
    print("\nPreparing visualization...")
    
    # Save all frames to a single multi-model PDB file
    with tempfile.NamedTemporaryFile(suffix='.pdb', delete=False, mode='w') as tmp:
        tmp_path = tmp.name
    
    # Write all frames as MODEL/ENDMDL blocks
    with open(tmp_path, 'w') as f:
        for frame_idx in range(traj_viz.n_frames):
            frame = traj_viz[frame_idx]
            frame_tmp = tmp_path + f".frame{frame_idx}.pdb"
            frame.save_pdb(frame_tmp, force_overwrite=True)
            with open(frame_tmp, 'r') as ff:
                content = ff.read()
            f.write(f"MODEL     {frame_idx + 1}\n")
            for line in content.split('\n'):
                if not line.startswith('MODEL') and not line.startswith('ENDMDL') and line.strip():
                    f.write(line + '\n')
            f.write("ENDMDL\n")
            Path(frame_tmp).unlink()
    
    # Read the multi-model PDB
    with open(tmp_path, 'r') as f:
        pdb_content = f.read()
    Path(tmp_path).unlink()
    
    # Create viewer
    view = py3Dmol.view(width=800, height=600)
    view.addModelsAsFrames(pdb_content, 'pdb')
    
    # Style - apply to all frames
    aa_list = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU', 
               'GLY', 'HIS', 'HID', 'HIE', 'HIP', 'ILE', 'LEU', 'LYS', 
               'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
    
    view.setStyle({'resn': aa_list}, {'cartoon': {'color': 'spectrum'}})
    
    # Style all ligands with different colors and add labels
    lig_colors = ['green', 'cyan', 'magenta', 'orange']
    for i, resn in enumerate(sorted(ligand_resnames)):
        color = lig_colors[i % len(lig_colors)]
        view.setStyle({'resn': resn}, {'stick': {'color': color, 'radius': 0.3}})
        view.addResLabels({'resn': resn}, {
            'fontSize': 12,
            'fontColor': 'white',
            'backgroundColor': color,
            'backgroundOpacity': 0.8
        })
    
    view.zoomTo()
    view.setProjection('orthographic')
    
    # Enable frame-based animation
    view.animate({'loop': 'forward', 'reps': 0, 'interval': 100})
    
    print(f"\n🔹 Protein: Cartoon (spectrum)")
    print(f"🔹 Ligands: {list(ligand_resnames)} (stick, different colors)")
    print(f"🔹 Frames: {traj_viz.n_frames} (animated)")
    print(f"\n▶️ Animation should auto-play")
    
    view.show()
else:
    print("Trajectory file not found. Run simulation first.")

## Summary

Complete MD workflow from PDB structure to trajectory visualization:

| Step | Description | Tool/Method |
|------|-------------|-------------|
| 1 | Fetch structure | `fetch_molecules` (RCSB PDB) |
| 2 | Prepare complex | `prepare_complex` (PDBFixer + antechamber) |
| 3 | Check merged | Auto-merged by prepare_complex |
| 4 | Visualize complex | py3Dmol |
| 5 | Solvate | `solvate_structure` (packmol-memgen) |
| 6 | Visualize solvated | py3Dmol |
| 7 | Build Amber system | `build_amber_system` (tleap) |
| 8 | Run MD | OpenMM (NPT) |
| 9 | Visualize trajectory | MDTraj + py3Dmol |


In [ ]:
# Print summary of generated files
print("=" * 60)
print("GENERATED FILES SUMMARY")
print("=" * 60)

print(f"\n📁 Output directory: {output_dir}")

print(f"\n📄 Protein files:")
for p in complex_result["proteins"]:
    if p["success"]:
        print(f"   • {Path(p['output_file']).name}")

print(f"\n📄 Ligand files:")
for lig in complex_result["ligands"]:
    if lig["success"]:
        print(f"   • {lig['ligand_id']}:")
        print(f"     - MOL2: {Path(lig['mol2_file']).name}")
        print(f"     - FRCMOD: {Path(lig['frcmod_file']).name}")

print(f"\n📄 Structure files:")
print(f"   • Merged complex: {Path(merged_pdb).name}")
print(f"   • Solvated system: {Path(solvated_pdb).name}")

print(f"\n📄 Amber files:")
if 'amber_result' in dir() and amber_result.get('success'):
    print(f"   • Topology: {Path(amber_result['parm7']).name}")
    print(f"   • Coordinates: {Path(amber_result['rst7']).name}")

print(f"\n📄 Simulation files:")
if 'dcd_file' in dir():
    print(f"   • Trajectory: {dcd_file.name}")
    print(f"   • Log: {log_file.name}")
    print(f"   • Final state: {final_pdb.name}")

print(f"\n✅ Workflow complete!")
